# Setup

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "results"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

# Helper functioins and structures
import helpers
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Use of machine learning techniques for heavenly bodies classification based on Sloan Digital Sky Survey

## Wprowadzenie
W tym notebooku spróbujemy przeprowadzić klasyfikację obiektów niebieskich jako gwiazdę, galaktykę lub kwazar. W tym celu spróbujemy kompletnego pojjścia jakim kierują się ludzie z dziłu tzw. Data Science. Na to podejście będzie się analiza danych, wybór cech, proces uczenia maszynowego wraz z selekcją cech oraz analiza otrzymanych wyników.

Cały proces zostanie przeprowadzony na zbiorze Sloan Digital Sky Survey.
### O zbiorze danych

Sloan Digital Sky Survey jest projektem, który oferuje publicznie dostępne dane o obserwacjach obiektów niebieskich. Dane te były zbierane od 1998r. i zostały udostępnione dla każdego, kto był zainteresowany.

Dla tego celu wykorzystany został specjalny teleskop od średnicy 2.5m, który zbudowano w Nowym Meksyku w obserwatorium Apache Point Observatory w USA. Teleskop używa kamery składającej się z 30 chipów CCD o rozdzielczości 2048x2048 każdy. Chipy są ułożone w 5 rzędów po 6 w każdym.Każdy rząd obserwuje przestrzeń kosmiczną przez zróżne optyczne filtry (u, g, r, i, z) w różnych długościach fal - 354, 476, 628, 769, 925 nm.

Teleskop pokrywa swym zasięgiem około 1/4 ziemskiego nieba - przez co skupia swój wzork na północnej części nieba.


## Analiza danych

### Zawartość
Zbiór składa się z 10 tys. obserwacji ciał niebieskich zebranych przez SDSS. Każda obserwacja jest opisana przez 17 atrybutów i jednej klasy przynależności - gwiazda, galaktyka lub kwazar.

### Opis cech

Zbiór powstał poprzez wykonaie kwerendy, która łączy ze sobą dwa widoki SQL. "PhotoObj", który zawiera fotometryczne dane oraz "SpecObj" zawierający dane spektralne.

View "PhotoObj"<br>
* **objid = Object Identifier**<br>
* **ra = J2000 Right Ascension (r-band)**<br>
* **dec = J2000 Declination (r-band)**<br>

Rektascensja, α (łac. recta ascensio – "wznoszenie proste") – jedna ze współrzędnych astronomicznych, określających położenie ciała niebieskiego na sferze niebieskiej w układzie współrzędnych astronomicznych zwanym układem równikowym równonocnym.

Definiujemy ją jako kąt dwuścienny pomiędzy płaszczyzną koła godzinnego punktu równonocy wiosennej (rektascensja równa 0h) a płaszczyzną koła godzinnego obiektu. Rektascensję nalicza się w kierunku na wschód, zgodnym z rocznym ruchem Słońca. Przyjmuje ona wartości z zakresu od 0h do 24h.

Współrzędna ta nie ulega zmianie na skutek ruchu obrotowego Ziemi, jak w przypadku kąta godzinnego. Zmianę współrzędnych, w tym deklinacji, powoduje precesja (ruch punktu Barana).

Deklinacja (łac. declinatio – „odchylenie”, oznaczana symbolem δ) – jedna ze współrzędnych określających położenie ciała w obydwu układach równikowych: równonocnym i godzinnym. Definiujemy ją jako kąt pomiędzy kierunkiem poprowadzonym od obserwatora do obiektu a płaszczyzną równika niebieskiego. Obiekty położone na północnej półkuli nieba mają deklinację dodatnią (od 0° do 90°), a na południowej ujemną (od 0° do -90°).

Zjawisko precesji skutkuje powolną zmianą współrzędnych astronomicznych (w tym rektascensji). Dlatego też oprócz nich podaje się również epokę, dla której zostały one wyznaczone (np. J2000.0).

Source: https://en.wikipedia.org/wiki/Right_ascension

u = better of DeV/Exp magnitude fit
g = better of DeV/Exp magnitude fit
r = better of DeV/Exp magnitude fit
i = better of DeV/Exp magnitude fit
z = better of DeV/Exp magnitude fit
The Thuan-Gunn astronomic magnitude system. u, g, r, i, z represent the response of the 5 bands of the telescope.

Further education: https://www.astro.umd.edu/~ssm/ASTR620/mags.html
<br>
* **run = Run Number**<br>
* **rereun = Rerun Number**<br>
* **camcol = Camera column**<br>
* **field = Field number**<br>

run, rerun, camcol oraz field są cechami, które opisują pole wewnątrz obrazu wykonanego przez SDSS. Pole jest po porstu częścią całego obrazu odpowiadającego 2048x1489 pikseli. Pole może być identyfikowane przez: 
* run number - identyfikator skanu,
* camcol - kolumna kamery, numer od 1 do 6, które identyfikujące linię skanu oraz 
* field - numer pola. Numer pola zazwyczaj zaczyna się na 11 (po początkowym okresie wznoszenia) i może osiągnąć wartość 800 dla wyjątkowo długich obserwacji.
* dodatkowo numer rerun, oznacza jak obraz był przetwarzany

View "SpecObj"<br>
* **specobjid = Object Identifier**<br>
* **class = object class (galaxy, star or quasar object)**<br>
Class identyfikuje objekt jako gwiazdę, galaktykę lub kawazar. Jest to zmienna, którą będziemy chcieli przewidzieć.
<br>
* **redshift = Final Redshift**<br>
* **plate = plate number**<br>
* **mjd = MJD of observation**<br>
* **fiberid = fiber ID**<br>

W fizyce, redshift (przesunięcie ku czerwieni) występuje, kiedy śiwatło lub inne promieniowanie elektormagnetyczne obiektu jest podwyższone w długości fali lub przesunięte w stronę czerwonego końca spektrum.

Każda ekspozycja spektoskopiczna wykorzystuje dużą, cieńką, okrągłą metalową płytę, która pozycjonuje światłowody przez dziury wywiercone w lokacjach obrazów w centrum skupienia teleskopu. Światłowdy te, następnie przekazują dane do spektrografów. Każda płyta ma unikalny numer seryjny, który jest wykorzystywany w widokach takich jak "SpecObj".  
mjd jest Zmodyfikowaną Datą Julianową, która określa dzień w którym dokonano pomiaru danej próbki SDSS.

Spektograf SDSS wykorzystuje światłowody do przekazywania światła do płaszczyzny ogniskowej z poszczególnych obiektów do "szczeliny"?? Każdy obiekt ma przypisane odpowiadające mu fiberID. 

Więcej informacji o SDSS i atrybutach:

http://www.sdss3.org/dr9/imaging/imaging_basics.php

http://www.sdss3.org/dr8/glossary.php